# Librerías



In [1]:
import os
import sys
from pathlib import Path
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd

notebook_dir = os.getcwd() 
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)

from config.config import get_db_config

# Constantes Globales

In [2]:
DDL_COMMANDS = {
    'CREATE',
    'DROP',
    'ALTER'
}

DML_COMMANDS = {
    'INSERT',
    'UPDATE',
    'DELETE'
}

# Funciones

In [ ]:
def get_create_table_as(query: str, schema: str, table_name: str) -> str:
    """
    Generate a CREATE TABLE AS query.

    Args:
        query (str): The base query to create the table from.
        schema (str): The schema name for the new table.
        table_name (str): The name of the new table.

    Returns:
        str: The complete CREATE TABLE AS query.
    """
    create_table_as_query = []

    create_table_as_query.append(f'CREATE TABLE {schema}.{table_name} AS (')
    create_table_as_query.extend(query.replace(';', '').splitlines())
    create_table_as_query.append(');')
    
    return '\n'.join(create_table_as_query)



def execute_commands(commands: list[str]) -> None:
    """
    Execute a list of SQL DDL/DML commands against the database.

    This function takes a list of SQL commands and executes them sequentially
    against the database. It ensures that only valid DDL (Data Definition Language)
    and DML (Data Manipulation Language) commands are executed.

    Args:
        commands (list[str]): A list of SQL commands to execute. Each command
                              must be a valid DDL or DML command.

    Raises:
        AssertionError: If any command in the list is not a valid DDL or DML command.
        psycopg2.DatabaseError: If there is a database error during execution.
        Exception: For any other exceptions that occur during execution.

    Returns:
        None
    """

    not_valid_commands = len([
        True 
        for command in commands 
        if command.strip().upper().split()[0] not in DDL_COMMANDS | DML_COMMANDS
    ])

    assert not_valid_commands == 0, f'The commands list contains {not_valid_commands} invalid DDL/DML commands.'
    
    config = get_db_config()

    try:
        with psycopg2.connect(**config) as connection:
            with connection.cursor() as cursor:
                for command in commands:
                    cursor.execute(command)
                    print(f'\nThe following command was executed sucessfully:\n{command}')

    except (psycopg2.DatabaseError, Exception) as e:
        print(e)



def execute_query(query: str) -> list[dict] | None:
    """
    Executes a SQL SELECT query and returns the results as a list of dictionaries.

    Args:
        query (str): The SQL SELECT query to execute.

    Returns:
        list[dict] | None: A list of dictionaries representing the query results,
                           or None if the query fails.

    Raises:
        AssertionError: If the query is not a SELECT command.
        psycopg2.DatabaseError: If there is a database error.
        Exception: For any other exceptions that occur.
    """
    assert query.strip().upper().split()[0] in ('SELECT', 'WITH'), 'The query must be a `SELECT` command.'

    query_results = None
    config = get_db_config()

    try:
        with psycopg2.connect(**config) as connection:
            with connection.cursor(cursor_factory=RealDictCursor) as cursor:
                cursor.execute(query)
                query_results = cursor.fetchall()

    except (psycopg2.DatabaseError, Exception) as e:
        print(e)

    return query_results


# Creación de esquemas

## Esquema de Tabla Plana (Flat Table)

Para crear este esquema, utilizaremos el script almacenado en el fichero `/data/database/postgres/reduced_schemas/flat_table.sql`. Adicionalmente, debemos ejecutar algunos comandos previos para asegurarnos la existencia de un nuevo esquema en blanco.

In [ ]:
FLAT_TABLE_SCHEMA = 'flat_table'
FLAT_TABLE_NAME = 'full_sales_data'
FLAT_TABLE_QUERY_PATH = '../data/database/postgres/reduced_schemas/flat_table/flat_table.sql'

commands_flat_table = []
commands_flat_table.append(f'DROP SCHEMA IF EXISTS {FLAT_TABLE_SCHEMA} CASCADE;')
commands_flat_table.append(f'CREATE SCHEMA {FLAT_TABLE_SCHEMA};')

flat_table_query = Path(FLAT_TABLE_QUERY_PATH).read_text()
commands_flat_table.append(get_create_table_as(flat_table_query, FLAT_TABLE_SCHEMA, FLAT_TABLE_NAME))

execute_commands(commands_flat_table)

query_resulst = execute_query(f'SELECT * FROM {FLAT_TABLE_SCHEMA}.{FLAT_TABLE_NAME} LIMIT 10')

display(pd.DataFrame(query_resulst))


The following command was executed sucessfully:
DROP SCHEMA IF EXISTS flat_table CASCADE;

The following command was executed sucessfully:
CREATE SCHEMA flat_table;

The following command was executed sucessfully:
CREATE TABLE flat_table.full_sales_data AS (
WITH
    all_sales AS (
        (
            SELECT
                product_key,
                -- order_date_key,
                -- due_date_key,
                -- ship_date_key,
                NULL AS reseller_key,
                NULL AS employee_key,
                customer_key,
                promotion_key,
                -- currency_key,
                sales_territory_key,
                sales_order_number,
                sales_order_line_number,
                -- revision_number,
                order_quantity,
                unit_price,
                extended_amount,
                unit_price_discount_pct,
                discount_amount,
                product_standard_cost,
                total_product_

,sale_source,sales_order_number,sales_order_line_number,order_quantity,unit_price,extended_amount,unit_price_discount_pct,discount_amount,product_standard_cost,total_product_cost,...,reseller_business_type,reseller_name,reseller_product_line,employee_key,employee_full_name,sales_territory_city,sales_territory_state_province,sales_territory_region,sales_territory_country,sales_territory_group
0,reseller_sales,SO44124,14,1,20.1865,20.1865,0.0,0.0,12.0278,12.0278,...,Warehouse,Advanced Bike Components,Road,283,Jillian Carson,Irving,Texas,United States - Southwest,Estados Unidos,North America
1,reseller_sales,SO45568,11,3,20.1865,60.5595,0.0,0.0,12.0278,36.0834,...,Warehouse,Advanced Bike Components,Road,283,Jillian Carson,Irving,Texas,United States - Southwest,Estados Unidos,North America
2,reseller_sales,SO46377,2,1,20.1865,20.1865,0.0,0.0,12.0278,12.0278,...,Warehouse,Advanced Bike Components,Road,283,Jillian Carson,Irving,Texas,United States - Southwest,Estados Unidos,North America
3,reseller_sales,SO43913,10,5,20.1865,100.9325,0.0,0.0,12.0278,60.1390,...,Value Added Reseller,Journey Sporting Goods,Mountain,283,Jillian Carson,Laredo,Texas,United States - Southwest,Estados Unidos,North America
4,reseller_sales,SO44566,13,2,20.1865,40.3730,0.0,0.0,12.0278,24.0556,...,Value Added Reseller,Journey Sporting Goods,Mountain,283,Jillian Carson,Laredo,Texas,United States - Southwest,Estados Unidos,North America
5,reseller_sales,SO46103,1,2,20.1865,40.3730,0.0,0.0,12.0278,24.0556,...,Value Added Reseller,Journey Sporting Goods,Mountain,283,Jillian Carson,Laredo,Texas,United States - Southwest,Estados Unidos,North America
6,reseller_sales,SO46099,13,2,20.1865,40.3730,0.0,0.0,12.0278,24.0556,...,Warehouse,Every Bike Shop,Road,285,Tsvi Reiter,La Vergne,Tennessee,United States - Southeast,Estados Unidos,North America
7,reseller_sales,SO44129,16,2,20.1865,40.3730,0.0,0.0,12.0278,24.0556,...,Warehouse,Larger Cycle Shop,Road,281,Michael Blythe,Melville,New York,United States - Northeast,Estados Unidos,North America
8,reseller_sales,SO44797,32,3,20.1865,60.5595,0.0,0.0,12.0278,36.0834,...,Warehouse,Larger Cycle Shop,Road,281,Michael Blythe,Melville,New York,United States - Northeast,Estados Unidos,North America
9,reseller_sales,SO45575,17,2,20.1865,40.3730,0.0,0.0,12.0278,24.0556,...,Warehouse,Larger Cycle Shop,Road,281,Michael Blythe,Melville,New York,United States - Northeast,Estados Unidos,North America
